<a href="https://colab.research.google.com/github/aknip/Local_LLMs/blob/main/Zephyr_7B_Alpha_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick start to run Zephyr 7B Alpha on Google Colab

-  A Low-Cost LLM Can Perform Better Than Llama-70B
- There is an immediate way to play with the Zephyr-7B model. HuggingFace has created a space for inference. https://huggingfaceh4-zephyr-chat.hf.space/

- Source: https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha/blob/main/colab-demo.ipynb

- Info: https://levelup.gitconnected.com/zephyr-7b-%CE%B1-a-low-cost-llm-can-perform-better-than-llama-70b-a2055dd028cb

other:
- https://github.com/aigeek0x0/zephyr-7b-alpha-langchain-chatbot

In [2]:
# Install transformers from source - only needed for versions <= v4.34
%%capture --no-stderr
%pip install git+https://github.com/huggingface/transformers.git
%pip install accelerate
# or !pip install transformers accelerate

## Load dependencies

In [25]:
import re
import torch
import textwrap
from transformers import pipeline

## Download model and load pipeline

takes approx. 3 mins

In [4]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# Helper functions

In [ ]:
def extract_response_text(input_text):
  input_text = outputs[0]["generated_text"]
  match = re.search(r'\<\|assistant\|\>(.+)', input_text, re.DOTALL)
  if match:
      extracted_text = match.group(1).strip()
  else:
      extracted_text = input_text.strip()

  return extracted_text

## Prepare inputs

In [5]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant". For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate.",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>



## Generate!

takes approx 40 secs on Colab T4

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
response_text = extract_response_text(outputs[0]["generated_text"])
print(textwrap.fill(response_text, 80))